# Environment

In [1]:
import numpy as np

class Env:
    
    def __init__(self):
         self.board = np.zeros((6,7))
         self.winner = None
         self.ended = False
         self.red = 1
         self.black = -1
         self.state_size = 3**(6*7)

    def is_empty(self, i ,j):
         return self.board[i][j] == 0


    def reward(self,color):
         if not self.game_over():
           return(0)
         if self.winner == color:
           return(1)
         elif self.winner == None:
           return(0)
         else:
           return(-1)

    def get_state(self):
         #####################
         # Use 3-base number #
         #####################
         state = 0
         for i in range(6):
            for j in range(7):
                if self.board[i,j] == 0:
                    u = 0
                elif self.board[i,j] == self.red:
                    u = 1
                else:
                    u = 2
                state +=  3**(3*i + j)*u
         return (state)

    def game_over(self):      
        
         for i in range(6):
            for l in range(5):
                if np.sum(self.board[i,l:(l+4)]) == self.red*4:
                   self.winner = 1
                   self.ended = True
                   return(True)
                elif np.sum(self.board[i,l:(l+4)]) == self.black*4:
                   self.winner = -1
                   self.ended = True
                   return(True)
         for j in range(7):
            for m in range(4):
                if np.sum(self.board[m:(m+4),j]) == self.red*4:
                   self.winner = 1
                   self.ended = True
                   return(True)
                elif np.sum(self.board[m:(m+4),j]) == self.black*4:
                   self.winner = -1
                   return(True)
         for i in range(6):
            for j in range(7):
                diag_sum = 0
                s,t = i,j
                while(s<=5 and s<=i+3 and t<=6 and t<=j+3):
                      diag_sum += self.board[s,t]
                      s += 1
                      t += 1
                if diag_sum == self.red*4:
                   self.winner =  1
                   self.eneded = True 
                   return(True)
                elif diag_sum == self.black*4:
                   self.winner = -1
                   self.ended = True 
                   return(True)

                ss, tt = i,j  
                diag_sum = 0
                while (ss<=5 and ss<=i+3 and tt>=0 and tt>=j-3):
                       diag_sum += self.board[ss,tt]
                       ss +=1
                       tt -=1
                if diag_sum == self.red*4:
                   self.winner = 1
                   self.ended = True
                   return(True)
                elif diag_sum == self.black*4:
                   self.winner = -1
                   self.ended = True
                   return(True)
                   
                
         ##################
         # check if draw  #   
         ##################
         if np.all((self.board == 0) == False):
             self.winner = None
             self.ended = True
             return(True)


         self.winner = None
         self.ended = False 
         return(False)



    def feasible_actions(self):

              feasible_actions = []

              for j in range(7):
                  for i in list(range(6))[::-1]:
                    if self.is_empty(i,j):
                       feasible_actions.append([i,j])
                       break       
                    
              return(feasible_actions) 


    def draw_board(self):
        for i in range(6):
            print('---------------------------')
            for j in range(7):
              print('  ', end='')
              if self.board[i,j] == self.red:
                  print('R ', end='')
              elif self.board[i,j] == self.black:
                  print('B ', end='')
              else:
                  print('  ', end='')
            print('')
        print('---------------------------')

    def reset_env(self):
         self.board = np.zeros((6,7))
         self.winner = None
         self.ended = False



# Deep Convolutional Residual Neural Network

In [2]:
from keras.models import Sequential, load_model, Model 
from keras.layers import Input, Dense, Conv2D, Flatten, BatchNormalization, Activation, LeakyReLU, add
from keras.optimizers import SGD 
from keras import regularizers 
import keras.backend as K
from keras import regularizers
import pandas as pd
import random 
from keras.models import clone_model 
import math

Using TensorFlow backend.
c:\users\chris bai\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\chris bai\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\chris bai\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint1

In [3]:
################################################################################
# See https://github.com/AppliedDataSciencePartners/DeepReinforcementLearning #
###############################################################################
import tensorflow as tf

def softmax_cross_entropy_with_logits(y_true, y_pred):

      p = y_pred
      pi = y_true
      zero = tf.zeros(shape = tf.shape(pi), dtype=tf.float32)
      where = tf.equal(pi, zero)
      negatives = tf.fill(tf.shape(pi), -100.0) 
      p = tf.where(where, negatives, p)

      loss = tf.nn.softmax_cross_entropy_with_logits(labels = pi, logits = p)

      return(loss)


class Gen_Model():
    
    def __init__(self, reg_const, learning_rate, input_dim, output_dim):
            
            self.reg_const = reg_const
            self.learning_rate = learning_rate 
            self.input_dim = input_dim 
            self.output_dim = output_dim 
            
    def predict(self, x):
            
             return(self.model.predict(x))
            
    def fit(self, states, targets, epochs, verbose, validation_split, batch_size):
          
             return(self.model.fit(states, targets, epochs = epochs, verbose = verbose, validation_split = validation_split, batch_size=batch_size))

class Residual_CNN(Gen_Model):
    
    def __init__(self,reg_const, learning_rate, input_dim, output_dim, hidden_layers):
            
            Gen_Model.__init__(self, reg_const, learning_rate, input_dim, output_dim)
            self.hidden_layers = hidden_layers
            self.num_layers = len(hidden_layers)
            self.model = self._build_model()
    
    

    def conv_layer(self, x, filters, kernel_size):
        
          x = Conv2D(
              filters = filters
              , kernel_size = kernel_size
              , data_format = "channels_first"
              , padding = 'same'
              , use_bias = False
              , activation = 'linear'
              , kernel_regularizer = regularizers.l2(self.reg_const)
              )(x)
            
          x = BatchNormalization(axis=1)(x)
        
          x = LeakyReLU()(x)
        
          return(x)
            
    
    def residual_layer(self, input_block, filters, kernel_size):
        
          x = self.conv_layer(input_block, filters, kernel_size)
        
          x = Conv2D(
                filters = filters
                , kernel_size = kernel_size
                , data_format = "channels_first"
                , padding = 'same'
                , use_bias = False
                , activation = 'linear'
                , kernel_regularizer = regularizers.l2(self.reg_const)
                    )(x)
    
          x = BatchNormalization(axis=1)(x)
        
        
          x = add([input_block, x])
          x = LeakyReLU()(x)
        
          return(x)
        
        
        
    def value_head(self, x):
         
            x = Conv2D(
            filters = 1 
            , kernel_size = (1,1)
            , data_format = "channels_first"
            , padding = 'same'
            , use_bias = False
            , activation = 'linear'
            , kernel_regularizer = regularizers.l2(self.reg_const)
            )(x)
            
            x = BatchNormalization(axis=1)(x)
            x = LeakyReLU()(x)
            
            x = Flatten()(x)
            
            x = Dense(
                   20
                   , use_bias = False 
                   , activation = 'linear'
                   )(x)
            
            x = LeakyReLU()(x)
            
            
            x = Dense(
                    1
                    , use_bias = False 
                    , activation = 'tanh'
                    , name = 'value_head'
                    )(x)
            
            
            return(x)
        
        
    
    def policy_head(self,x):
        
            x = Conv2D(
            filters = 2
            , kernel_size = (1,1)
            , data_format = "channels_first"
            , padding = 'same'
            , use_bias = False 
            , activation = 'linear'
            )(x)
            
            x = BatchNormalization(axis=1)(x)
            x = LeakyReLU()(x)
            
            x = Flatten()(x)
            
            x = Dense(
                  self.output_dim
                  , use_bias= False 
                  , activation = 'linear'
                  , kernel_regularizer = regularizers.l2(self.reg_const)
                  , name = 'policy_head'
            )(x)
            
            return(x)
            

          
    def _build_model(self):
        
           main_input = Input(shape = self.input_dim, name = 'main_input')
            
           x = self.conv_layer(main_input, self.hidden_layers[0]['filters'], self.hidden_layers[0]['kernel_size'])
        
           if len(self.hidden_layers) > 1:
                   for h in self.hidden_layers[1:]:
                        x = self.residual_layer(x, h['filters'], h['kernel_size'] )
                        
           vh = self.value_head(x)
           ph = self.policy_head(x)
            
           
           model = Model(inputs = [main_input], outputs = [vh,ph])
           
           model.compile(loss = {'value_head': 'mean_squared_error', 'policy_head': softmax_cross_entropy_with_logits},
                         optimizer = SGD(lr=self.learning_rate),
                         loss_weights = {'value_head': 0.5, 'policy_head': 0.5}
                                 ) 
            
           return(model)            

In [4]:
hidden_layers = [
    {'filters':75, 'kernel_size': (4,4)}
  , {'filters':75, 'kernel_size': (4,4)}
  , {'filters':75, 'kernel_size': (4,4)}
  , {'filters':75, 'kernel_size': (4,4)}
  , {'filters':75, 'kernel_size': (4,4)}
  , {'filters':75, 'kernel_size': (4,4)}
  ]

In [5]:
m_1 = Residual_CNN(reg_const = 0.0001, learning_rate = 0.1, input_dim = (1,6,7), output_dim = (42), hidden_layers = hidden_layers)

W0820 15:32:49.612427 16124 deprecation_wrapper.py:119] From c:\users\chris bai\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0820 15:32:49.668466 16124 deprecation_wrapper.py:119] From c:\users\chris bai\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0820 15:32:49.672930 16124 deprecation_wrapper.py:119] From c:\users\chris bai\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0820 15:32:49.704705 16124 deprecation_wrapper.py:119] From c:\users\chris bai\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:174: The name tf.get_default_s

# Monte Carlo Tree Search

In [6]:
class Node:
    
    def __init__(self):
        
          self.parent = None 
          self.children = []
          self.n_i = 0 
          self.v_i = 0
          self.state = np.zeros((6,7))
        
    def get_children(self):
          
          return(self.children)
                      

In [7]:
class MCTS:
    
    def __init__(self, C, symbol, res_cnn_model):
        
          self.C = C 
          self.symbol = symbol 
          self.res_cnn_model = res_cnn_model 
        
    def get_prior_idx(self, current_state, child_state):
          
          e = Env()
          e.board = child_state - current_state
          pos = np.where(e.board != 0) 
          row = pos[0][0]
          col = pos[1][0]
          prior_idx = row*7+col
          
          return(prior_idx)
            
        
    
    def traversal(self, current_node):
            
          while len(current_node.get_children()) > 0:
              
             current_state = current_node.state
             ph = self.res_cnn_model.predict(current_state.reshape((1,1,6,7)))[1][0]
             
                
             #UCB = [node.V_i + self.C*ph[self.get_prior_idx(current_node.state,node.state)]*(np.sqrt(current_node.n_i)/node.n_i) 
             #           for node in current_node.get_children]
             
             UCB = [self.res_cnn_model.predict(node.state.reshape((1,1,6,7)))[0][0] 
                    + self.C*ph[self.get_prior_idx(current_node.state,node.state)]*(np.sqrt(current_node.n_i)/(node.n_i+1)) 
                        for node in current_node.get_children()] 
                
                
             next_node = np.argmax(UCB)
             current_node = current_node.get_children()[next_node]
                
          return(current_node)
    
    def p_play(self, state):
        
        
          e = Env()
          e.board = state.copy()
          count_O = (np.array(state) ==1).sum()
          count_X = (np.array(state) == -1).sum()
          
          if count_O < count_X:
            
             sym = 1 
                
          else:
            
            
             sym = -1
                
          while not e.game_over():
            
             action = None
            
             ph = self.res_cnn_model.predict(e.board.reshape((1,1,6,7)))[1][0]
             
             feasible_actions = e.feasible_actions()
             feasible_actions_nbr = len(feasible_actions)
               
             p_feasible = []
            
             action_tmp = []
            
             for act in feasible_actions:
                    
                    old_board = e.board.copy()
                    e.board[act[0], act[1]] = sym 
                    if e.game_over():
                            action_tmp.append(act)
                    e.board[act[0], act[1]] = 0
            
             action_nbr = len(action_tmp)
                
             if action_nbr>0:
                    
                    idx = np.random.randint(action_nbr)
                    action = action_tmp[idx]
            
             if action == None:
                    
                action_tmp = []
            
                for act in feasible_actions:
                    
                    old_board = e.board.copy()
                    e.board[act[0], act[1]] = sym*(-1)
                    if e.game_over():
                        action_tmp.append(act)
                    e.board[act[0],act[1]] = 0
                    
                action_nbr = len(action_tmp)
                
                if action_nbr>0:
                    
                    idx = np.random.randint(action_nbr)
                    action = action_tmp[idx]
                        
             if action == None:            
                            
                
                for act in feasible_actions:
                    
                    old_board = e.board.copy()
                    e.board[act[0],act[1]] = sym 
                    pp = ph[self.get_prior_idx(old_board, e.board)]            
                    
                    p_feasible.append(pp)
                    
                    e.board[act[0],act[1]] = 0
                    
                p_all = sum(p_feasible)
                s = [i for i in range(feasible_actions_nbr)]
             
                if p_all == 0:
                       idx = random.choices(s)[0]
                    
                else:
                       p_feasible_scaled = [p/(p_all*1.0) for p in p_feasible]
                       idx = random.choices(s,p_feasible_scaled)[0]
             
                action = feasible_actions[idx]       
                    
             e.board[action[0], action[1]] = sym 
            
             sym = sym*(-1)
                
        
          return(e.reward(self.symbol))              
             
    
    def rollout(self, current_node):
        
          current_node = current_node 
            
          e_temp = Env()
        
          e_temp.board = current_node.state.copy()
            
          if current_node.n_i == 0 or e_temp.game_over():
            
             V = self.p_play(current_node.state)
             current_node.v_i = (current_node.v_i*current_node.n_i + V)/(current_node.n_i + 1)
             current_node.n_i = current_node.n_i + 1
                
             return(current_node)
        
        
          else:
                
                e = Env()
                e.board = current_node.state.copy()
                count_O = (np.array(e.board) == 1).sum()
                count_X = (np.array(e.board) == -1).sum()
                
                if count_O < count_X:
                    
                    sym = 1
                
                else:
                    
                    sym = -1
                    
                feasible_actions = e.feasible_actions()
                
                for act in feasible_actions:
                    
                    e.board[act[0],act[1]] = sym
                    new_node = Node()
                    new_node.parent = current_node
                    new_node.state = e.board.copy()
                    e.board[act[0], act[1]] = 0
                    current_node.children.append(new_node)
                    
                child_node = current_node.children[0]
                
                V = self.p_play(child_node.state)
                
                child_node.v_i = (child_node.v_i*child_node.n_i + V)/(child_node.n_i+1)
                child_node.n_i = child_node.n_i + 1
                
            
          return(child_node) 
    
    
    def backpropagation(self, current_node):
        
          play_hist = dict(s_hist = [], v_hist = [], p_hist = [])
            
          p = [0]*42
        
          children = current_node.get_children()
        
          if len(children)>0:
                
                play_hist['state_hist'].append(current_node.state)
                play_hist['v_hist'].append(current_node.v_i)
                
                for child in children:
                    
                    idx = self.get_prior_idx(current.state, child.state)
                    p[idx] = child.n_i/(current_node.n_i*1.0)
                
                play_hist['p_hist'].append(p)
                
                
            
          while current_node.parent is not None:
            
                p = [0]*42
                 
                current_node.parent.v_i = (current_node.parent.v_i*current_node.parent.n_i + current_node.v_i)/(current_node.parent.n_i + 1)
                current_node.parent.n_i = current_node.parent.n_i + 1
                
                play_hist['s_hist'].append(current_node.parent.state)
                play_hist['v_hist'].append(current_node.parent.v_i)
                
                children = current_node.parent.get_children()
                 
                for child in children:
                    
                    idx = self.get_prior_idx(current_node.parent.state, child.state)
                    p[idx] = child.n_i/(current_node.parent.n_i*1.0)
                    
                play_hist['p_hist'].append(p)  
                current_node = current_node.parent
                
          return(play_hist)        

# Iterations

In [8]:
def get_prior_idx(current_state, child_state):
    
          e = Env()
          e.board = child_state - current_state
          pos = np.where(e.board != 0) 
          row = pos[0][0]
          col = pos[1][0]
          prior_idx = row*7+col
          
          return(prior_idx)
        
def model_eval(m1):
   
       e = Env()

       while not e.game_over():
        
        # m_1 for black 
        # m_2 for red 
        
         action_black  = None
        
         p_black = m1.predict(e.board.reshape((1,1,6,7)))[1][0]
        
         feasible_actions_black = e.feasible_actions()
         feasible_actions_black_nbr = len(feasible_actions_black)
         p_black_feasible = []
            
         action_tmp = []   
            
         for act in feasible_actions_black:
                    
                    old_board = e.board.copy()
                    e.board[act[0], act[1]] = -1 
                    if e.game_over():
                            action_tmp.append(act) 
                    e.board[act[0], act[1]] = 0
                    
         action_nbr = len(action_tmp)
         
         if action_nbr>0:
                    
                    idx = np.random.randint(action_nbr)
                    action_black = action_tmp[idx]
         
                    
         if action_black == None:
                  
                    action_tmp = []
            
                    for act in feasible_actions_black:
                    
                         old_board = e.board.copy()
                         e.board[act[0], act[1]] = 1
                         if e.game_over():
                             action_tmp.append(act)
                         e.board[act[0],act[1]] = 0
                    
                    action_nbr = len(action_tmp)
                
                    if action_nbr>0:
                
                         idx = np.random.randint(action_nbr)
                         action_black = action_tmp[idx]
                        
         if action_black == None: 
        
                    for act in feasible_actions_black:
                    
                      old_board = e.board.copy()
                      e.board[act[0],act[1]] = -1 
                      pp_black = p_black[get_prior_idx(e.board, old_board)]
                    
                      p_black_feasible.append(pp_black)
                      e.board[act[0], act[1]] = 0
             
                    p_black_all = sum(p_black_feasible)
                    s_black = [i for i in range(feasible_actions_black_nbr)]
            
                    if p_black_all == 0:
                
                        idx_black = random.choices(s_black)[0]
                    
                    else:
                    
                        p_black_feasible_scaled = [p/(p_black_all*1.0) for p in p_black_feasible]
        
                        idx_black = random.choices(s_black, p_black_feasible_scaled)[0]
         
                    action_black = feasible_actions_black[idx_black]
            
         e.board[action_black[0], action_black[1]] = -1 
        
         #############################################################################
            
         feasible_actions_red = e.feasible_actions()
          
         feasible_actions_nbr = len(feasible_actions_red)
        
         
         idx = np.random.randint(feasible_actions_nbr)
         action = feasible_actions_red[idx]
         e.board[action[0], action[1]] = 1
       
         
         #print(e.board)
     
       
       if e.winner == -1:           

           return('model')
       
       elif e.winner == None:
                
           return('draw')
        
       else:
           
           return('random')          

# Iteration 0

In [9]:
root = Node()
s_train = []
v_train = []
p_train = []
M = MCTS(C = 15, symbol= -1, res_cnn_model = m_1)

In [10]:
for i in range(5000):
  current_node = M.traversal(root)
  end_node = M.rollout(current_node)
  play_hist = M.backpropagation(end_node)
  s_train = s_train + play_hist['s_hist']
  v_train = v_train + play_hist['v_hist']
  p_train = p_train + play_hist['p_hist']
  if i % 500 == 0: 
       print(i)

0
500
1000
1500
2000
2500
3000
3500
4000
4500


In [11]:
s_train_tail = s_train[(len(s_train)-8000):len(s_train)]
p_train_tail = p_train[(len(s_train)-8000):len(s_train)]
v_train_tail = v_train[(len(v_train)-8000):len(v_train)]

In [12]:
s_nbr = len(s_train_tail)
s_nbr

8000

In [13]:
s_train_1 = np.array(s_train_tail).reshape((s_nbr,1,6,7))
p_train_1 = np.array(p_train_tail).reshape((s_nbr,42))
v_train_1 = np.array(v_train_tail).reshape((s_nbr,1))

In [14]:
m_1.fit(s_train_1,[v_train_1, p_train_1] , epochs = 1, verbose = True, validation_split = 0, batch_size = 256)

Epoch 1/1
8000/8000 [==============================] - 34s 4ms/step - loss: 0.4493 - value_head_loss: 0.2169 - policy_head_loss: 0.5195


In [15]:
m_iter_0 = m_1

Evaluate model

In [16]:
result = []

for i in range(2000):

     result.append(model_eval(m_1))
     #if i % 100 ==  0:
     #    print(i)
            
result_df = {'cnt' : result}
result_df = pd.DataFrame(result_df)
result_df.cnt.value_counts(dropna = False)

model     1917
random      82
draw         1
Name: cnt, dtype: int64

# Iteration 1

In [25]:
#root = Node()
#s_train = []
#v_train = []
#p_train = []
#M = MCTS(C = 15, symbol= -1, res_cnn_model = m_1)

In [26]:
#for i in range(500):
#  current_node = M.traversal(root)
#  end_node = M.rollout(current_node)
#  play_hist = M.backpropagation(end_node)
#  s_train = s_train + play_hist['s_hist']
#  v_train = v_train + play_hist['v_hist']
#  p_train = p_train + play_hist['p_hist']
#  if i % 100 == 0: 
#       print(i)

In [27]:
#s_nbr = len(s_train)
#s_nbr
#s_train_2 = np.array(s_train).reshape((s_nbr,1,6,7))
#p_train_2 = np.array(p_train).reshape((s_nbr,42))
#v_train_2 = np.array(v_train).reshape((s_nbr,1))

In [28]:
#m_1.fit(s_train_2,[v_train_2, p_train_2] , epochs = 1, verbose = True, validation_split = 0, batch_size = 256)

Evaluate model

In [29]:
#result = []

#for i in range(1000):
#
#     result.append(model_eval(m_1))
     #if i % 100 ==  0:
     #    print(i)
#result_df = {'cnt' : result}
#result_df = pd.DataFrame(result_df)
#result_df.cnt.value_counts(dropna = False)

In [30]:
#m_iter_1 = m_1

# Iteration 2

In [31]:
#root = Node()
#s_train = []
#v_train = []
#p_train = []
#M = MCTS(C = 15, symbol= -1, res_cnn_model = m_1)

In [32]:
#for i in range(500):
#  current_node = M.traversal(root)
#  end_node = M.rollout(current_node)
#  play_hist = M.backpropagation(end_node)
#  s_train = s_train + play_hist['s_hist']
#  v_train = v_train + play_hist['v_hist']
#  p_train = p_train + play_hist['p_hist']
#  if i % 100 == 0: 
#       print(i)

In [33]:
#s_nbr = len(s_train)
#s_nbr
#s_train_2 = np.array(s_train).reshape((s_nbr,1,6,7))
#p_train_2 = np.array(p_train).reshape((s_nbr,42))
#v_train_2 = np.array(v_train).reshape((s_nbr,1))

In [34]:
#m_1.fit(s_train_2,[v_train_2, p_train_2] , epochs = 1, verbose = True, validation_split = 0.2, batch_size = 256)

Evaluate Model

In [35]:
#result = []

#for i in range(1000):

#     result.append(model_eval(m_1))
     #if i % 100 ==  0:
     #    print(i)
#result_df = {'cnt' : result}
#result_df = pd.DataFrame(result_df)
#result_df.cnt.value_counts(dropna = False)

In [36]:
#m_iter_2 = m_1

# Play

In [17]:
class Human:

    def __init__(self):
        pass

    def usr_play(self,env):

        action = [int(x) for x in input().split()]

        while action not in env.feasible_actions():
            print("This is not a legitimate action! Please input a another action.")
            action = [int(x) for x in input().split()]
        
        env.board[action[0], action[1]] = 1

In [22]:
#############
#  Play!    #
#############

def play(Env, H, m):

      print("Do you want to play? Yes/No")
      ans = input ()
      while ans.lower() == 'yes':
         
        while not Env.game_over():
            
           action_black = None  
            
           p_black = m.predict(Env.board.reshape((1,1,6,7)))[1][0]
        
           feasible_actions_black = Env.feasible_actions()
           feasible_actions_black_nbr = len(feasible_actions_black)
           p_black_feasible = []
            
           action_tmp = []   
            
           for act in feasible_actions_black:
                    
                    old_board = Env.board.copy()
                    Env.board[act[0], act[1]] = -1 
                    if Env.game_over():
                            action_tmp.append(act) 
                    Env.board[act[0], act[1]] = 0
                    
           action_nbr = len(action_tmp)
         
           if action_nbr>0:
                    
                    idx = np.random.randint(action_nbr)
                    action_black = action_tmp[idx]
                    
           if action_black == None:
                    
                    action_tmp = []
            
                    for act in feasible_actions_black:
                    
                         old_board = Env.board.copy()
                         Env.board[act[0], act[1]] = 1
                         if Env.game_over():
                             action_tmp.append(act)
                         Env.board[act[0],act[1]] = 0
                    
                    action_nbr = len(action_tmp)
                
                    if action_nbr>0:
                
                         idx = np.random.randint(action_nbr)
                         action_black = action_tmp[idx]
                        
           if action_black == None:
        
                    for act in feasible_actions_black:
                    
                         old_board = Env.board.copy()
                         Env.board[act[0],act[1]] = -1 
                         pp_black = p_black[get_prior_idx(Env.board, old_board)]
                    
                         p_black_feasible.append(pp_black)
                         Env.board[act[0], act[1]] = 0
             
                    p_black_all = sum(p_black_feasible)
                    s_black = [i for i in range(feasible_actions_black_nbr)]
            
                    if p_black_all == 0:
                
                         idx_black = random.choices(s_black)[0]
                    
                    else:
                    
                         p_black_feasible_scaled = [p/(p_black_all*1.0) for p in p_black_feasible]
        
                         idx_black = random.choices(s_black, p_black_feasible_scaled)[0]
         
                    action_black = feasible_actions_black[idx_black]
            
           Env.board[action_black[0], action_black[1]] = -1 
        
           Env.draw_board()
           
           if not Env.game_over():
              
              print("Please take an action!")
              H.usr_play(Env)
              Env.draw_board()
                
        if Env.winner == -1:
            print('Game over! The winner is {0}'.format('Black'))
        elif Env.winner == 1:
            print('Game over! The winner is {0}'.format('Red'))
        else:
            print('Game over! It is a draw!')
        Env.reset_env()
        print("Do you want to play again? Yes/No")
        ans = input()

      print("Session ended!")

In [24]:
e = Env()
H = Human()
play(e,H,m_1)

Do you want to play? Yes/No
Yes
---------------------------
                            
---------------------------
                            
---------------------------
                            
---------------------------
                            
---------------------------
                            
---------------------------
              B             
---------------------------
Please take an action!
5 0
---------------------------
                            
---------------------------
                            
---------------------------
                            
---------------------------
                            
---------------------------
                            
---------------------------
  R           B             
---------------------------
---------------------------
                            
---------------------------
                            
---------------------------
                            
---------------------------
  

4 1
---------------------------
                            
---------------------------
                            
---------------------------
                            
---------------------------
          R           B   B 
---------------------------
      R   B           R   B 
---------------------------
  B   R   B           R   R 
---------------------------
---------------------------
                            
---------------------------
                            
---------------------------
                            
---------------------------
      B   R           B   B 
---------------------------
      R   B           R   B 
---------------------------
  B   R   B           R   R 
---------------------------
Please take an action!
2 1
---------------------------
                            
---------------------------
                            
---------------------------
      R                     
---------------------------
      B   R           B   B 
-

4 5
---------------------------
                            
---------------------------
                            
---------------------------
                            
---------------------------
                            
---------------------------
      B               R     
---------------------------
      R   R   B   B   R   B 
---------------------------
---------------------------
                            
---------------------------
                            
---------------------------
                            
---------------------------
      B                     
---------------------------
      B               R     
---------------------------
      R   R   B   B   R   B 
---------------------------
Please take an action!
4 3
---------------------------
                            
---------------------------
                            
---------------------------
                            
---------------------------
      B                     
-

1 0
---------------------------
      B                   R 
---------------------------
  R   B                   R 
---------------------------
  R   R           B   B   R 
---------------------------
  B   B       R   R   B   B 
---------------------------
  R   B   B   R   B   R   R 
---------------------------
  B   R   R   B   B   R   B 
---------------------------
---------------------------
      B                   R 
---------------------------
  R   B                   R 
---------------------------
  R   R           B   B   R 
---------------------------
  B   B   B   R   R   B   B 
---------------------------
  R   B   B   R   B   R   R 
---------------------------
  B   R   R   B   B   R   B 
---------------------------
Please take an action!
2 3
---------------------------
      B                   R 
---------------------------
  R   B                   R 
---------------------------
  R   R       R   B   B   R 
---------------------------
  B   B   B   R   R   B   B 
-

1 0
---------------------------
                            
---------------------------
  R               R         
---------------------------
  B               R         
---------------------------
  B               B         
---------------------------
  B   R       R   B   B   B 
---------------------------
  R   R       B   R   R   B 
---------------------------
---------------------------
                            
---------------------------
  R               R         
---------------------------
  B               R         
---------------------------
  B           B   B         
---------------------------
  B   R       R   B   B   B 
---------------------------
  R   R       B   R   R   B 
---------------------------
Please take an action!
2 3
---------------------------
                            
---------------------------
  R               R         
---------------------------
  B           R   R         
---------------------------
  B           B   B         
-